In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
a = pd.read_csv('final.csv', index_col=False)
b = pd.read_csv('radiomics.csv', index_col=False)

C:\Users\anton\AppData\Local\Temp\ipykernel_2536\2453495840.py:2: DtypeWarning: Columns (38,39,40,41,47) have mixed types. Specify dtype option on import or set low_memory=False.
  b = pd.read_csv('radiomics.csv', index_col=False)


In [7]:
a.head()

,Id,patient_id,slice_thickness,pixel_spacing,subtlety,internalStructure,calcification,sphericity,margin,lobulation,spiculation,texture,diameter,surface_area,volume,malignancy
0,LIDC-IDRI-0001-1,LIDC-IDRI-0001,2.5,0.70,5,1,6,3,3,3,4,5,32.205888,2801.428431,7869.369507,1
1,LIDC-IDRI-0001-2,LIDC-IDRI-0001,2.5,0.70,5,1,6,4,4,5,5,5,33.188692,2198.454852,6146.438599,1
2,LIDC-IDRI-0001-3,LIDC-IDRI-0001,2.5,0.70,5,1,6,3,2,3,3,5,31.959332,2392.704188,6594.474792,1
3,LIDC-IDRI-0001-4,LIDC-IDRI-0001,2.5,0.70,5,1,6,5,4,1,5,4,33.669337,2573.278820,7348.411560,1
4,LIDC-IDRI-0002-1,LIDC-IDRI-0002,1.2,0.68,2,1,6,5,1,1,1,1,30.023172,2880.758652,6576.900681,1


In [8]:
b.head()

,id,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,...,logarithm_gldm_LargeDependenceLowGrayLevelEmphasis,logarithm_gldm_LowGrayLevelEmphasis,logarithm_gldm_SmallDependenceEmphasis,logarithm_gldm_SmallDependenceHighGrayLevelEmphasis,logarithm_gldm_SmallDependenceLowGrayLevelEmphasis,logarithm_ngtdm_Busyness,logarithm_ngtdm_Coarseness,logarithm_ngtdm_Complexity,logarithm_ngtdm_Contrast,logarithm_ngtdm_Strength
0,LIDC-IDRI-0001-1,v3.0.1,1.26.0,2.3.0,1.4.1,3.11.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",536a24a547656a8d43436937029df0d651b44259,3D,...,0.315204,0.038768,0.437417,662.328267,0.014300,0.143802,0.002087,36576.623708,6.768040,14.090091
1,LIDC-IDRI-0001-2,v3.0.1,1.26.0,2.3.0,1.4.1,3.11.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",536a24a547656a8d43436937029df0d651b44259,3D,...,0.030970,0.009428,0.439498,682.658274,0.006310,0.099596,0.002513,31985.177011,5.824263,14.843045
2,LIDC-IDRI-0001-3,v3.0.1,1.26.0,2.3.0,1.4.1,3.11.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",536a24a547656a8d43436937029df0d651b44259,3D,...,0.051793,0.013446,0.478543,777.359773,0.007703,0.117973,0.002163,39403.439570,6.843639,14.682214
3,LIDC-IDRI-0001-4,v3.0.1,1.26.0,2.3.0,1.4.1,3.11.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",536a24a547656a8d43436937029df0d651b44259,3D,...,0.103967,0.018713,0.463454,710.169563,0.008608,0.134292,0.002153,35828.160710,7.225162,14.078258
4,LIDC-IDRI-0002-1,v3.0.1,1.26.0,2.3.0,1.4.1,3.11.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",536a24a547656a8d43436937029df0d651b44259,3D,...,75.851878,0.455616,0.104524,126.985909,0.012662,0.792276,0.007417,3154.963087,0.988412,55.765308
